# The Outer Solar System

---


## Data

The chosen units are: masses relative to the sun, so that the sun has mass $1$. We have taken $m_0 = 1.00000597682$ to take account of the inner planets. Distances are in astronomical units , times in earth days, and the gravitational constant is thus $G = 2.95912208286 \cdot 10^{−4}$.

| planet | mass | initial position | initial velocity |
| --- | --- | --- | --- |
| Jupiter | $m_1 = 0.000954786104043$ | <ul><li>−3.5023653</li><li>−3.8169847</li><li>−1.5507963</li></ul> | <ul><li>0.00565429</li><li>−0.00412490</li><li>−0.00190589</li></ul>
| Saturn | $m_2 = 0.000285583733151$ | <ul><li>9.0755314</li><li>−3.0458353</li><li>−1.6483708</li></ul> | <ul><li>0.00168318</li><li>0.00483525</li><li>0.00192462</li></ul>
| Uranus | $m_3 = 0.0000437273164546$ | <ul><li>8.3101420</li><li>−16.2901086</li><li>−7.2521278</li></ul> | <ul><li>0.00354178</li><li>0.00137102</li><li>0.00055029</li></ul>
| Neptune | $m_4 = 0.0000517759138449$ | <ul><li>11.4707666</li><li>−25.7294829</li><li>−10.8169456</li></ul> | <ul><li>0.00288930</li><li>0.00114527</li><li>0.00039677</li></ul>
| Pluto | $ m_5 = 1/(1.3 \cdot 10^8 )$ | <ul><li>−15.5387357</li><li>−25.2225594</li><li>−3.1902382</li></ul> | <ul><li>0.00276725</li><li>−0.00170702</li><li>−0.00136504</li></ul>

The data is taken from the book "Geometric Numerical Integration" by E. Hairer, C. Lubich and G. Wanner.

In [ ]:
using Plots, OrdinaryDiffEq, ForwardDiff
gr()

In [2]:
G = 2.95912208286e-4
M = Diagonal(repeat([1.00000597682, 0.000954786104043, 0.000285583733151,
        0.0000437273164546, 0.0000517759138449, 1/1.3e8], inner=3))       # repeat each mass three time
                                                                          # becasue there are 3 degrees
                                                                          # of freedom for each planet/sun

invM = inv(M).diag    # the diagonal of the inverse of the mass matrix
                      # will explain it later

planets = ["Sun", "Jupiter", "Saturn", "Uranus", "Neptune", "Pluto"]

pos = [
    0.,0.,0.,                              -3.5023653,-3.8169847,-1.5507963,
    9.0755314,-3.0458353,-1.6483708,       8.3101420,-16.2901086,-7.2521278,
    11.4707666,-25.7294829,-10.8169456,    -15.5387357,-25.2225594,-3.1902382
]
vel = [
    0,0,0,                               0.00565429,-0.00412490,-0.00190589,
    0.00168318,0.00483525,0.00192462,    0.00354178,0.00137102,0.00055029,
    0.00288930,0.00114527,0.00039677,    0.00276725, -0.00170702, -0.00136504
]

tspan = (0.,200_000)

(0.0, 200000)

The N-body problem's Hamiltonian is

$$H(p,q) = \frac{1}{2}\sum_{i=0}^{N}\frac{p_{i}^{T}p_{i}}{m_{i}} - G\sum_{i=1}^{N}\sum_{j=0}^{i-1}\frac{m_{i}m_{j}}{\left\lVert q_{i}-q_{j} \right\rVert}$$

Here, we want to solve for the motion of the five outer planets relative to the sun, namely, Jupiter, Saturn, Uranus, Neptune and Pluto.

In [3]:
const ∑ = sum
N = 5

ø = i -> (3i+1):(3(i+1))    # Generate indices like 1:3, 4:6 ...
T(p) = ∑(i->(p[ø(i)]'p[ø(i)])/M[3i+1,3i+1], 0:N)/2
V(q) = -G*∑(i->∑(j->(M[3i+1,3i+1]*M[3j+1,3j+1])/norm(q[ø(i)]-q[ø(j)]), 0:i-1), 1:N)
function Hamiltonian(p,q)
    T(p) + V(q)
    T+V
end

Hamiltonian (generic function with 1 method)

## Hamiltonian System

$$\dot{p} = -H_{q}(p,q)\quad \dot{q}=H_{p}(p,q)$$

We can symplify $\dot{q}$ as

$$\dot{p} = -\nabla{V}(q)\quad \dot{q}=M^{-1}p$$

because only potential energy depends on $q$ in this case. So, we can write a second order ODE as

$$\ddot{q} = f(q) = -M^{-1}\nabla{V}(q)$$

In [4]:
function acceleration(t, x, v, dv)
    ForwardDiff.gradient!(dv, V, x)
    for i in eachindex(dv)
        dv[i] *= -invM[i]
    end
end

acceleration (generic function with 1 method)

In [5]:
prob = SecondOrderODEProblem(acceleration, pos, vel, tspan)
@time sol = solve(prob, Yoshida6(), dt=100);

  5.841762 seconds (13.09 M allocations: 1.572 GiB, 4.72% gc time)


In [6]:
function plot_orbits(sol,N,planets)
    p = plot(sol, vars=(1,2,3), title="The Orbits of the Outer Solar System", lab=planets[1])
    for i in 1:N
        plot!(p, sol, vars=(ø(i)...), lab=planets[i+1])
    end
    p
end

plot_orbits (generic function with 1 method)

In [8]:
plot_orbits(sol, N, planets)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip00)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 135.433,268.398 135.433,268.398 132.142,271.241 135.433,268.398 135.433,253.605 133.788,255.026 135.433,253.605 135.433,238.811 132.142,241.655 135.433,238.811 
 135.433,224.018 133.788,225.44 135.433,224.018 135.433,209.225 132.142,212.069 135.433,209.225 135.433,194.432 133.788,195.854 135.433,194.432 135.433,179.639 
 132.142,182.482 135.433,179.639 135.433,164.846 133.788,166.267 135.433,164.846 135.433,150.052 132.142,152.896 135.433,150.052 
 "/>
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip00)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 135.433,268.398 135.878,268.625 134.233,270.047 135.878,268.625 162.029,281.975 158.738,284.819 162.029,281.975 188.179,295.325 186.534,296.747 188.179,295.325 
 214.329,308.675 211.039,311.519 214.329,308.675 240.48,322.025 238.834,323.447 240.48,322.025 266.63,335.375 263.339,338.219 266.63,335.375 292.781,348.725 
 291.135,350.147 292.781,348.725 318.931,362.075 315.64,364.919 318.931,362.075 331.524,368.504 
 "/>
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip00)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 331.524,368.504 332.857,367.352 334.818,368.353 332.857,367.352 354.791,348.398 358.713,350.4 354.791,348.398 376.726,329.445 378.687,330.446 376.726,329.445 
 398.66,310.491 402.582,312.493 398.66,310.491 420.595,291.537 422.556,292.538 420.595,291.537 442.529,272.584 446.451,274.586 442.529,272.584 464.464,253.63 
 466.425,254.631 464.464,253.63 486.399,234.676 490.32,236.679 486.399,234.676 496.063,226.325 
 "/>
 
 The Orbits of the Outer Solar System 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 281.466,191.49 281.466,191.49 281.466,191.49 281.466,191.49 281.466,191.49 281.466,191.49 281.466,191.49 281.466,191.49 281.466,191.49 281.466,191.49 
 281.466,191.49 281.466,191.49 281.466,191.49 281.466,191.49 281.466,191.49 281.466,191.49 281.466,191.49 281.466,191.49 281.466,191.49 281.466,191.49 
 281.466,191.49 281.466,191.49 281.466,191.49 281.466,191.49 281.465,191.49 281.465,191.49 281.465,191.49 281.465,191.49 281.465,191.49 281.465,191.49 
 281.465,191.491 281.465,191.491 281.465,191.491 281.465,191.491 281.465,191.491 281.465,191.491 281.464,191.491 281.464,191.491 281.464,191.491 281.464,191.491 
 281.464,191.491 281.464,191.491 281.464,191.491 281.464,191.492 281.464,191.492 281.464,191.492 281.463,191.492 281.463,191.492 281.463,191.492 281.463,191.492 
 281.463,191.492 281.463,191.492 281.463,191.493 281.462,191.493 281.462,191.493 281.462,191.493 281.462,191.493 281.462,191.493 281.462,191.493 281.462,191.493 
 281.462,191.494 281.461,191.494 281.461,191.494 281.461,191.494 281.461,191.494 281.461,191.494 281.461,191.495 281.46,191.495 281.46,191.495 281.46,191.495 
 281.46,191.495 281.46,191.495 281.46,191.496 281.46,191.496 281.459,191.496 281.459,191.496 281.459,191.496 281.459,191.497 281.459,191.497 281.459,191.497 
 281.458,191.497 281.458,191.498 281.458,191.498 281.458,191.498 281.458,191.498 281.458,191.498 281.457,191.499 281.457,191.499 281.457,191.499 281.457,191.499 
 281.457,191.5 281.457,191.5 281.456,191.5 281.456,191.5 281.456,191.501 281.456,191.501 281.456,191.501 281.456,191.502 281.455,191.502 281.455,191.502 
 281.455,191.502 281.455,191.503 281.455,191.503 281.455,191.503 281.454,191.504 281.454,191.504 281.454,191.504 281.454,191.505 281.454,191.505 281.454,191.505 
 281.453,191.505 281.453,191.506 281.453,191.506 281.453,191.506 281.453,191.507 281.453,191.507 281.453,191.507 281.452,191.508 281.452,191.508 281.452,191.509 
 281.452,191.509 281.452,191.509 281.452,191.51 281.452,191.51 281.451,191.51 281.4